In [7]:
import sys
import os
from pathlib import Path
# This appends the directory one level up (the root of your project) to the sys.path.
# Modify the path depending on the location of modules you want to import.
sys.path.append(os.path.abspath('../../'))

from config.config_managers import DashboardConfigManager
from dataManager import DataManager
from dash import Dash
import pandas as pd
import plotly.express as px
from abc import ABC, abstractmethod
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [54]:
class Visualization(ABC):
    def __init__(self, data, mappings):
        self.data = data
        self.tag_mapping = mappings['tag_mapping']
        self.dataset_mapping = mappings['dataset_mapping']

    @abstractmethod
    def prepare_data(self):
        pass

    @abstractmethod
    def visualize(self):
        pass

    def replace_mappings(self, data):
        data = self.replace_tags(data)
        data = self.replace_model(data)
        # data['Model'] = data['Model'].replace(self.dataset_mapping)
        return data
    
    def replace_tags(self, data):
        data['Tag'] = data['Tag'].replace(self.tag_mapping)
        return data
    
    def replace_model(self, data):
        data['Model'] = data['Model'].replace(self.dataset_mapping)
        return data
    
    def replace_columns_and_index(self, data):
        return data.rename(columns=self.tag_mapping, index=self.tag_mapping)

        
    
    

mappings = {
    'tag_mapping': {'B-PERS': 'B-PER', 'I-PERS': 'I-PER',},
    'dataset_mapping': {'ANERCorp_CamelLab_arabertv02': 'AraBERTv02', 'conll2003_bert': 'BERT'}
}

class ReportBarChart(Visualization):
    def prepare_data(self):
        report_data = []
        for model_name, data_content in self.data.items():
            token_report = data_content.token_report
            token_report['Model'] = model_name
            report_data.append(pd.concat([
                token_report, 
            ]))
        report_df = pd.concat(report_data)
        report_df = report_df[~report_df['Tag'].isin(['accuracy/micro', 'macro', 'weighted'])]    
        
        return self.replace_mappings(report_df)

    def visualize(self):
        token_report_data = self.prepare_data()
        melted_df = token_report_data.melt(
        id_vars=["Tag", "Support", "Model"], 
        value_vars=["Precision", "Recall"], 
        var_name="Metric", 
        value_name="Value"
        )
        
        # Round the values for better readability
        melted_df['Value'] = melted_df['Value'].round(3)
        
        # Create the bar chart using Plotly Express
        fig = px.bar(
            melted_df, 
            x="Tag", 
            y="Value", 
            color="Metric",
            facet_row="Model",  # Separate rows for each model
            title="Breakdown of Precision and Recall Scores by Entity Tag, Categorized by Mode",
            labels={"Value": "Score", 'Tag': 'Entity'},
            barmode="group",
            template="plotly_white",
            facet_row_spacing=0.1,
            text='Value'  # Display the Value on top of each bar
        )
        fig.update_layout(
            width=1200,
            height=400,
            # margin=dict(t=50, l=50, r=50, b=50)
        )
        
        # Show the figure
        fig.show()
    
    def visualize_f1(self):
        token_report_data = self.prepare_data()
        token_report_data['F1'] = token_report_data['F1'].round(3)
        fig = px.bar(token_report_data,
                    x="Tag",
                    y="F1",
                    color="Model",
                    title="Breakdown of F1 Score Per Model",
                    barmode='group',
                    template="plotly_white",
                    text='F1',  # Display the actual Count on top of each bar
                    )
        
        fig.update_layout(
            width=1200,
            height=400,
            # margin=dict(t=50, l=50, r=50, b=50)
        )
                
        fig.show()
        
        
    def visualize_support(self):
        token_report_data = self.prepare_data()
        token_report_data = token_report_data[token_report_data['Tag'] !='O']
        fig = px.bar(token_report_data,
                    x="Tag", 
                    y="Support",
                    color="Model",
                    title="Breakdown of the Number of Examples Per Model",
                    barmode='group',
                    template="plotly_white",
                    # facet_row_spacing=0.1,  # Adjusted spacing
                    facet_col_spacing=0.08,
                    text='Support',  # Display the actual Count on top of each bar
                    )
        fig.update_layout(
            width=1200,
            height=400,
            # margin=dict(t=50, l=50, r=50, b=50)
        )
                
        fig.show()

class ConfusionBarChart(Visualization):
    def prepare_data(self):
        matrices = []
        for model_name, data_content in self.data.items():
            token_matrix = pd.DataFrame(data_content.token_confusion_matrix['confusion_matrix']).T 
            token_matrix['Model'] = model_name
            matrices.append(pd.concat([
                token_matrix, 
            ]))
        matrix_df = pd.concat(matrices)
        matrix_df.reset_index(inplace=True)
        matrix_df.rename(columns={'index': 'Tag'}, inplace=True)
        matrix_data = self.replace_mappings(matrix_df)
        
                
        grouped = matrix_data.groupby(['Tag', 'Model']).sum()
        grouped['Total'] = grouped['TP'] + grouped['FP'] + grouped['FN']
        
        matrix_data = matrix_data.merge(grouped['Total'], on=['Tag', 'Model'], how='left')
        
        matrix_data['TP_Count'] = matrix_data['TP']
        matrix_data['FP_Count'] = matrix_data['FP']
        matrix_data['FN_Count'] = matrix_data['FN']
        
        matrix_data['TP'] = matrix_data['TP'] / matrix_data['Total']
        matrix_data['FP'] = matrix_data['FP'] / matrix_data['Total']
        matrix_data['FN'] = matrix_data['FN'] / matrix_data['Total']
        
        confusion_scaled_df = matrix_data.melt(id_vars=["Tag", "Model"], value_vars=["TP", "FP", "FN"], var_name="Metric", value_name="Scale")
        confusion_count_df = matrix_data.melt(id_vars=["Tag", "Model"], value_vars=["TP_Count", "FP_Count", "FN_Count"], var_name="Metric", value_name="Count")
        confusion_count_df['Metric'] = confusion_count_df['Metric'].str.replace('_Count', '')
        confusion_data = confusion_scaled_df.merge(confusion_count_df, on=["Tag", "Model", "Metric"])
        return confusion_data

    def visualize(self):
        confusion_df = self.prepare_data()
        print(confusion_df)
        
        fig = px.bar(confusion_df,
                    x="Tag", y="Scale", color="Metric",
                    facet_row="Model",
                    title="Breakdown of Confusion Matrix Components: by Entity Tag, Categorized by Model",
                    labels={"Scale": "Scaled Counts"},
                    barmode='group',
                    template="plotly_white",
                    facet_row_spacing=0.1,  # Adjusted spacing
                    text='Count'  # Display the actual Count on top of each bar
                    )
        
        fig.show()



class ConfusionHeatmap(Visualization):
    def prepare_data(self):
        matrices = []
        for model_name, data_content in self.data.items():
            token_matrix = pd.DataFrame(data_content.token_confusion_matrix['confusion_matrix']).T 
            token_matrix['Model'] = model_name
            matrices.append(pd.concat([
                token_matrix, 
            ]))
        matrix_df = pd.concat(matrices)
        matrix_df.reset_index(inplace=True)
        matrix_df.rename(columns={'index': 'Tag'}, inplace=True)
        return self.replace_mappings(matrix_df)
        

    def visualize(self):
        token_matrix = self.prepare_data()
        token_matrix = token_matrix[token_matrix['Tag'] !='O']
        confusion_df = token_matrix.melt(
            id_vars=['Tag', 'Model'], 
            value_vars=['TP', 'FP', 'FN'], 
            var_name='Metric', 
            value_name='Count'
        )
        
        unique_models = confusion_df['Model'].unique()  # Only two models
        max_value = confusion_df['Count'].max()  # Get the max value for consistent color scaling

        # Create a subplot for each model
        fig = make_subplots(
            rows=len(unique_models), cols=1,  # Rows for models, single column
            subplot_titles=[f"{model}" for model in unique_models],
            shared_yaxes=True, horizontal_spacing=0.5, vertical_spacing=0.1
        )

        for idx, model in enumerate(unique_models):
            # Filter data for the current model
            filtered_data = confusion_df[confusion_df['Model'] == model]
            
            # Prepare heatmap data
            heatmap_data = filtered_data.pivot_table(index='Metric', columns='Tag', values='Count', fill_value=0)
            text_data = heatmap_data.astype(int)  # Convert values to integers for display

            # Add heatmap trace for the current model
            fig.add_trace(
                go.Heatmap(
                    z=heatmap_data.values,  # Heatmap values
                    x=heatmap_data.columns,  # Columns (Tags)
                    y=heatmap_data.index,  # Rows (Metrics)
                    colorscale='RdBu_r',
                    coloraxis="coloraxis",  # Unified color axis
                    text=text_data,  # Add text annotations
                    texttemplate="%{text}",  # Use the text values directly
                    hovertemplate="Metric: %{y}<br>Tag: %{x}<br>Count: %{text}<extra></extra>",
                ),
                row=idx + 1, col=1  # Stack rows for models
            )
        
        # Update layout with unified color axis and title
        fig.update_layout(
            coloraxis=dict(colorscale='RdBu_r', cmin=0, cmax=max_value, colorbar=dict(title="Counts")),
            title_text="Confusion Matrix Heatmap Categorized by Model",
            template="plotly_white",
            
            height=300 * len(unique_models),  # Adjust height dynamically based on the number of models
            width=800,  # Adjust the width if necessary
        )

        # Show the figure
        fig.show()
        

class ErrorTypeHeatmap(Visualization):
    def prepare_data(self):
        matrix_data = []
        # Step 1: Collect general error data
        for model_name, data_content in self.data.items():
            token_confusion = data_content.token_misclassifications
                
            # Annotate with Model and Scheme
            token_confusion['Model'] = model_name
            token_confusion = self.replace_columns_and_index(token_confusion)
            matrix_data.append(token_confusion)
        
        # Step 2: Combine and process data
        matrix_df = pd.concat(matrix_data)
        matrix_df.reset_index(inplace=True)
        matrix_df.rename(columns={'index': 'Tag'}, inplace=True)
        matrix_df = self.replace_model(matrix_df)
        return matrix_df

    def visualize(self):
        token_misclassifications = self.prepare_data()
        
        # Group by 'Model'
        models = token_misclassifications['Model'].unique()

        # Create subplots for heatmaps
        fig = make_subplots(
            rows=1,
            cols=len(models),
            subplot_titles=models,
            horizontal_spacing=0.2
        )
        max_value = token_misclassifications.drop(columns=['Tag', 'Model']).values.max()

        # Iterate over each model and create a heatmap
        for i, model in enumerate(models, start=1):
            model_df = token_misclassifications[token_misclassifications['Model'] == model].set_index('Tag')
            model_df = model_df.drop(columns='Model')  # Drop the Model column
            fig.add_trace(
                go.Heatmap(
                    z=model_df.values,
                    x=model_df.columns,
                    y=model_df.index,
                    colorscale='RdBu_r',
                    coloraxis="coloraxis",  # Unified color axis
                    text=model_df.values,
                    texttemplate="%{text}",
                    hovertemplate="True: %{y}<br>Predict: %{x}<br>Count: %{text}<extra></extra>",  # Updated hover labels
                ),
                row=1, col=i
            )
        for idx in range(1, len(models) + 1):
            fig.update_yaxes(title_text="True Labels", row=1, col=idx)
            fig.update_xaxes(title_text="Predicted Labels", row=1, col=idx)

        # Update layout
        fig.update_layout(
            coloraxis=dict(colorscale='RdBu_r', cmin=0, cmax=max_value, colorbar=dict(title="Counts")),
            title="Errors Heatmap Across Entity Tags, Categorized by Model",
            template="plotly_white",
            width=1200,
            height=600,
       
        )

        fig.show()
    
    def visualize_table(self):
        """
        Generates tables showing raw counts and percentages for each error type, 
        categorized by Scheme and Model.
        """
        # Step 1: Prepare the data
        token_misclassifications = self.prepare_data()
        print(token_misclassifications.to_string(index=False)) 

In [9]:
CONFIG_PATH = Path("/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/dashboard-config.yaml")
config_manager = DashboardConfigManager(CONFIG_PATH)
dev_config = config_manager.development_config    

app = Dash(__name__, suppress_callback_exceptions=True)

app_config = config_manager.app_config
server = app.server  # Flask server instance for caching
variants_data = None

data_manager = DataManager(config_manager, server)
dash_data = data_manager.load_data()

2024-12-31 20:02:17 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/analysis/My Drive
2024-12-31 20:02:17 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-12-31 20:02:17 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02


  0%|          | 0/18 [00:00<?, ?it/s]

2024-12-31 20:02:25 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/analysis/My Drive
2024-12-31 20:02:25 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-12-31 20:02:25 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/conll2003_bert


  0%|          | 0/18 [00:00<?, ?it/s]

In [38]:
tr_df = dash_data['ANERCorp_CamelLab_arabertv02'].train_data

In [12]:
report_data = []
for data_name, data_content in dash_data.items():
	token_report = data_content.token_report 
	tr_data = data_content.train_data
	token_report['Model'] = data_name
	tr_support = tr_data[tr_data['Labels']!= -100]['True Labels'].value_counts().reset_index().rename(columns = {'True Labels': 'Tag', 'count': 'Train Support'})
	tr = token_report.merge(tr_support, on='Tag')
	report_data.append(pd.concat([
		tr, 
	]))
report_df = pd.concat(report_data)
report_df = report_df[~report_df['Tag'].isin(['accuracy/micro', 'macro', 'weighted'])]    
tag_mapping = {
    'B-PERS': 'B-PER',
    'I-PERS': 'I-PER'
}

dataset_mapping = {
    'ANERCorp_CamelLab_arabertv02': 'AraBERTv02',
    'conll2003_bert': 'BERT'
}

report_df['Tag'] = report_df['Tag'].replace(tag_mapping)
report_df['Model'] = report_df['Model'].replace(dataset_mapping)
report_df


,Tag,Precision,Recall,F1,Support,Model,Train Support
0,B-LOC,0.9056,0.9476,0.9261,668,AraBERTv02,3772
1,B-MISC,0.8238,0.6766,0.7430,235,AraBERTv02,887
2,B-ORG,0.8237,0.7889,0.8059,450,AraBERTv02,1576
3,B-PER,0.8955,0.8788,0.8871,858,AraBERTv02,2719
4,I-LOC,0.8095,0.8193,0.8144,83,AraBERTv02,525
5,I-MISC,0.8193,0.4121,0.5484,165,AraBERTv02,375
6,I-ORG,0.8024,0.7236,0.7610,275,AraBERTv02,1115
7,I-PER,0.9188,0.9002,0.9094,641,AraBERTv02,2202
8,O,0.9857,0.9933,0.9895,21616,AraBERTv02,111488
0,B-LOC,0.9287,0.9371,0.9329,1668,BERT,7140


In [13]:
# Confusion matrix heatmap example
report_bar = ReportBarChart(dash_data, mappings)
report_bar.visualize_f1()

In [14]:
# Confusion matrix heatmap example
report_bar = ReportBarChart(dash_data, mappings)
report_bar.visualize()

In [15]:
# Confusion matrix heatmap example
report_bar = ReportBarChart(dash_data, mappings)
report_bar.visualize_support()

In [45]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Prepare data for Pearson and Spearman correlations
metrics = ["Precision", "Recall"]
pearson_data = []
spearman_data = []
data = report_df[report_df["Tag"] != "O"]  # Exclude "O" tag for analysis

for model in data["Model"].unique():
    model_data = data[data["Model"] == model]
    for metric in metrics:
        # Pearson correlation
        test_corr_pearson = model_data[["Support", metric]].corr().iloc[0, 1]
        train_corr_pearson = model_data[["Train Support", metric]].corr().iloc[0, 1]
        pearson_data.append({"Metric": metric, "Model": model, "Train": train_corr_pearson, "Test": test_corr_pearson})
        
        # Spearman correlation
        test_corr_spearman = model_data[["Support", metric]].corr(method="spearman").iloc[0, 1]
        train_corr_spearman = model_data[["Train Support", metric]].corr(method="spearman").iloc[0, 1]
        spearman_data.append({"Metric": metric, "Model": model, "Train": train_corr_spearman, "Test": test_corr_spearman})

# Create DataFrames for Pearson and Spearman
pearson_df = pd.DataFrame(pearson_data)
spearman_df = pd.DataFrame(spearman_data)

# Pivot data for heatmap format
pearson_train_heatmap = pearson_df.pivot(index="Metric", columns="Model", values="Train")
pearson_test_heatmap = pearson_df.pivot(index="Metric", columns="Model", values="Test")
spearman_train_heatmap = spearman_df.pivot(index="Metric", columns="Model", values="Train")
spearman_test_heatmap = spearman_df.pivot(index="Metric", columns="Model", values="Test")

# Create subplots for Pearson and Spearman
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Pearson Train Correlation", "Pearson Test Correlation", 
                    "Spearman Train Correlation", "Spearman Test Correlation"),
    horizontal_spacing=0.2,
    vertical_spacing=0.2,
    shared_xaxes=True,
    shared_yaxes=True,
)

# Add Pearson Train heatmap
fig.add_trace(
    go.Heatmap(
        z=pearson_train_heatmap.values,
        x=pearson_train_heatmap.columns,
        y=pearson_train_heatmap.index,
        text=pearson_train_heatmap.values.round(3),
        texttemplate="%{text}",
        hovertemplate="Metric: %{y}<br>Model: %{x}<br>Correlation: %{z}<extra></extra>",
        colorscale="RdBu_r",
        zmin=-1,
        zmax=1,
    ),
    row=1, col=1,
)

# Add Pearson Test heatmap
fig.add_trace(
    go.Heatmap(
        z=pearson_test_heatmap.values,
        x=pearson_test_heatmap.columns,
        y=pearson_test_heatmap.index,
        text=pearson_test_heatmap.values.round(3),
        texttemplate="%{text}",
        hovertemplate="Metric: %{y}<br>Model: %{x}<br>Correlation: %{z}<extra></extra>",
        colorscale="RdBu_r",
        zmin=-1,
        zmax=1,
    ),
    row=1, col=2,
)

# Add Spearman Train heatmap
fig.add_trace(
    go.Heatmap(
        z=spearman_train_heatmap.values,
        x=spearman_train_heatmap.columns,
        y=spearman_train_heatmap.index,
        text=spearman_train_heatmap.values.round(3),
        texttemplate="%{text}",
        hovertemplate="Metric: %{y}<br>Model: %{x}<br>Correlation: %{z}<extra></extra>",
        colorscale="RdBu_r",
        zmin=-1,
        zmax=1,
    ),
    row=2, col=1,
)

# Add Spearman Test heatmap
fig.add_trace(
    go.Heatmap(
        z=spearman_test_heatmap.values,
        x=spearman_test_heatmap.columns,
        y=spearman_test_heatmap.index,
        text=spearman_test_heatmap.values.round(3),
        texttemplate="%{text}",
        hovertemplate="Metric: %{y}<br>Model: %{x}<br>Correlation: %{z}<extra></extra>",
        colorscale="RdBu_r",
        zmin=-1,
        zmax=1,
    ),
    row=2, col=2,
)

# Update layout
fig.update_layout(
    title="Combined Pearson and Spearman Correlation Heatmaps Across Metrics and Models",
    template="plotly_white",
    height=800,
    width=1000,
    coloraxis=dict(colorbar=dict(title="Correlation")),
    margin=dict(t=50, l=50, r=50, b=50),
)

fig.show()


In [41]:
import pandas as pd
import plotly.express as px

# Filter out the 'O' tag
data = report_df[report_df["Tag"] != "O"]

# Melt the DataFrame for plotting
melted_df = data.melt(
    id_vars=["Tag", "Model", "Train Support", "Support"],
    value_vars=["Precision", "Recall"],
    var_name="Metric",
    value_name="Value"
)

# Create separate columns for Train and Test support
melted_train = melted_df.copy()
melted_train["Support Type"] = "Train"
melted_train["Support Value"] = melted_train["Train Support"]

melted_test = melted_df.copy()
melted_test["Support Type"] = "Test"
melted_test["Support Value"] = melted_test["Support"]

# Combine Train and Test data
combined_df = pd.concat([melted_train, melted_test])

# Scatter plot with Train and Test as colors
fig = px.scatter(
    combined_df,
    x="Support Value",
    y="Value",
    color="Support Type",  # Color by Train or Test
    facet_col="Metric",
    facet_row="Model",
    trendline="ols",  # Add trendline (linear regression here)
    facet_row_spacing=0.1,  # Adjusted spacing
    facet_col_spacing=0.03,
    hover_data=["Tag"],
    title="Scatter Plot of Train and Test Support vs Metrics Across Models",
    labels={
        "Support Value": "Support",
        "Value": "Metric Value",
        "Metric": "Metric",
        "Support Type": "Support Type",
    },
    template="plotly_white"
)

# Update layout for better readability
fig.update_layout(
    width=1000,
    height=600,
    xaxis_title="Support",
    yaxis_title="Metric Value",
    legend_title="Support Type",
    # margin=dict(t=50, l=50, r=50, b=50)
)

fig.show()


In [44]:
def calculate_pearson_contributions_with_summary(df, x_col, y_col):
    """
    Calculate the numerator (covariance contributions) and denominator (variance contributions) 
    for Pearson correlation for each data point, and include model averages and dataset correlation.
    
    Parameters:
    - df: DataFrame containing the data.
    - x_col: Column name for the x variable (e.g., Support).
    - y_col: Column name for the y variable (e.g., Precision).
    
    Returns:
    - DataFrame with covariance and variance contributions for each data point,
      and prints model averages and dataset correlation.
    """
    # Print model averages
    print("Model Averages:")
    model_means = df.groupby("Model")[[x_col, y_col]].mean()
    print(model_means)

    # Calculate dataset-wide correlation
    correlation = df[x_col].corr(df[y_col])
    print(f"\nDataset-wide Pearson Correlation ({x_col} vs {y_col}): {correlation:.4f}\n")

    # Calculate means
    x_mean = df[x_col].mean()
    y_mean = df[y_col].mean()

    # Print means
    print(f"Overall {x_col} Mean: {x_mean}")
    print(f"Overall {y_col} Mean: {y_mean}\n")

    # Calculate deviations
    df["X Deviation"] = df[x_col] - x_mean
    df["Y Deviation"] = df[y_col] - y_mean

    # Calculate contributions
    df["Covariance Contribution"] = df["X Deviation"] * df["Y Deviation"]
    df["X Variance Contribution"] = df["X Deviation"] ** 2
    df["Y Variance Contribution"] = df["Y Deviation"] ** 2
    df["Variance Contribution"] = df["X Variance Contribution"] * df["Y Variance Contribution"]

    return df[["Tag", "Model", x_col, y_col, "X Deviation", "Y Deviation", 
               "Covariance Contribution", "X Variance Contribution", "Y Variance Contribution"]]

# Apply the function
pearson_contributions = calculate_pearson_contributions_with_summary(data .copy(), "Train Support", "Recall")
pearson_contributions

Model Averages:
            Train Support    Recall
Model                              
AraBERTv02       1646.375  0.768388
BERT             4255.375  0.905787

Dataset-wide Pearson Correlation (Train Support vs Recall): 0.6630

Overall Train Support Mean: 2950.875
Overall Recall Mean: 0.8370875



,Tag,Model,Train Support,Recall,X Deviation,Y Deviation,Covariance Contribution,X Variance Contribution,Y Variance Contribution
0,B-LOC,AraBERTv02,3772,0.9476,821.125,0.110513,90.744577,6.742463e+05,0.012213
1,B-MISC,AraBERTv02,887,0.6766,-2063.875,-0.160488,331.226139,4.259580e+06,0.025756
2,B-ORG,AraBERTv02,1576,0.7889,-1374.875,-0.048187,66.251789,1.890281e+06,0.002322
3,B-PER,AraBERTv02,2719,0.8788,-231.875,0.041713,-9.672086,5.376602e+04,0.001740
4,I-LOC,AraBERTv02,525,0.8193,-2425.875,-0.017787,43.150252,5.884870e+06,0.000316
5,I-MISC,AraBERTv02,375,0.4121,-2575.875,-0.424987,1094.714677,6.635132e+06,0.180614
6,I-ORG,AraBERTv02,1115,0.7236,-1835.875,-0.113487,208.348864,3.370437e+06,0.012879
7,I-PER,AraBERTv02,2202,0.9002,-748.875,0.063113,-47.263373,5.608138e+05,0.003983
0,B-LOC,BERT,7140,0.9371,4189.125,0.100013,418.964864,1.754877e+07,0.010003
1,B-MISC,BERT,3438,0.8390,487.125,0.001912,0.931627,2.372908e+05,0.000004


In [32]:
import pandas as pd
from scipy.stats import rankdata

def calculate_spearman_contributions_by_model(df, x_col, y_col):
    """
    Calculate contributions to Spearman correlation for each data point within each model,
    including rank differences and squared differences.
    
    Parameters:
    - df: DataFrame containing the data.
    - x_col: Column name for the x variable (e.g., Support).
    - y_col: Column name for the y variable (e.g., Precision or Recall).
    
    Returns:
    - DataFrame with rank differences and contributions for each data point,
      and prints Spearman correlation for each model.
    """
    # Initialize a list to collect results
    results = []

    # Group by 'Model' and calculate contributions within each group
    for model, group in df.groupby('Model'):
        # Rank the columns within the group
        group["X Rank"] = rankdata(group[x_col])
        group["Y Rank"] = rankdata(group[y_col])

        # Calculate rank differences
        group["Rank Difference"] = group["X Rank"] - group["Y Rank"]

        # Calculate squared rank differences
        group["Squared Rank Difference"] = group["Rank Difference"] ** 2

        # Calculate Spearman correlation for the group
        spearman_corr = group["X Rank"].corr(group["Y Rank"])
        print(f"Spearman Correlation for {model}: {spearman_corr:.4f}")

        # Append the group's results to the list
        results.append(group)

    # Concatenate all results into a single DataFrame
    result_df = pd.concat(results)

    return result_df[["Tag", "Model", x_col, y_col, "X Rank", "Y Rank", "Rank Difference", "Squared Rank Difference"]]

# Example usage
# Assuming 'data' is your DataFrame containing the necessary columns


Spearman Correlation for AraBERTv02: 0.7143
Spearman Correlation for BERT: 0.8095


,Tag,Model,Train Support,Precision,X Rank,Y Rank,Rank Difference,Squared Rank Difference
0,B-LOC,AraBERTv02,3772,0.9056,8.0,7.0,1.0,1.0
1,B-MISC,AraBERTv02,887,0.8238,3.0,5.0,-2.0,4.0
2,B-ORG,AraBERTv02,1576,0.8237,5.0,4.0,1.0,1.0
3,B-PER,AraBERTv02,2719,0.8955,7.0,6.0,1.0,1.0
4,I-LOC,AraBERTv02,525,0.8095,2.0,2.0,0.0,0.0
5,I-MISC,AraBERTv02,375,0.8193,1.0,3.0,-2.0,4.0
6,I-ORG,AraBERTv02,1115,0.8024,4.0,1.0,3.0,9.0
7,I-PER,AraBERTv02,2202,0.9188,6.0,8.0,-2.0,4.0
0,B-LOC,BERT,7140,0.9287,8.0,6.0,2.0,4.0
1,B-MISC,BERT,3438,0.8296,3.0,2.0,1.0,1.0


In [ ]:
train_spearman_contributions = calculate_spearman_contributions_by_model(data, "Train Support", "Recall")
train_spearman_contributions['Split'] = 'Train'
test_spearman_contributions = calculate_spearman_contributions_by_model(data, "Support", "Precision")
test_spearman_contributions['Split'] = 'Test'
spearman_contributions_combined = pd.concat([train_spearman_contributions, test_spearman_contributions])

Spearman Correlation for AraBERTv02: 0.8333
Spearman Correlation for BERT: 0.6667
Spearman Correlation for AraBERTv02: 0.7143
Spearman Correlation for BERT: 0.7619


In [46]:
# Plot using Plotly
fig = px.bar(
    spearman_contributions_combined,
    x="Tag",
    y="Squared Rank Difference",
    color="Model",
    facet_col="Split",
    text = 'Squared Rank Difference',
    title="Squared Rank Difference by Tag, Model, and Split",
    labels={"Squared Rank Difference": "Squared Rank Diff.", "Tag": "Entity Tag"},
    template="plotly_white",
    barmode="group",
    # height=500,
    # width=1000,
)

# Enhance layout
fig.update_layout(
    xaxis_title="Entity Tag",
    yaxis_title="Squared Rank Difference",
    legend_title="Model",
    margin=dict(t=50, l=50, r=50, b=50)
)

# Display the chart
fig.show()

In [41]:
confusion_bar = ConfusionBarChart(dash_data, mappings)
confusion_bar.visualize()

       Tag       Model Metric     Scale  Count
0   B-MISC  AraBERTv02     TP  0.591078    159
1   I-MISC  AraBERTv02     TP  0.377778     68
2    B-ORG  AraBERTv02     TP  0.674905    355
3        O  AraBERTv02     TP  0.979249  21472
4    I-LOC  AraBERTv02     TP  0.686869     68
5    B-LOC  AraBERTv02     TP  0.862398    633
6    I-ORG  AraBERTv02     TP  0.614198    199
7    B-PER  AraBERTv02     TP  0.797040    754
8    I-PER  AraBERTv02     TP  0.833815    577
9   B-MISC        BERT     TP  0.715674    589
10  I-MISC        BERT     TP  0.557093    161
11   B-ORG        BERT     TP  0.839118   1523
12   B-PER        BERT     TP  0.933573   1560
13       O        BERT     TP  0.990688  38088
14   I-LOC        BERT     TP  0.792642    237
15   B-LOC        BERT     TP  0.874161   1563
16   I-ORG        BERT     TP  0.840000    777
17   I-PER        BERT     TP  0.977797   1145
18  B-MISC  AraBERTv02     FP  0.126394     34
19  I-MISC  AraBERTv02     FP  0.083333     15
20   B-ORG  A

In [47]:
# Confusion matrix heatmap example
confusion_heatmap = ConfusionHeatmap(dash_data, mappings)
confusion_heatmap.visualize()

In [55]:
error_type_heatmap = ErrorTypeHeatmap(dash_data, mappings)
error_type_heatmap.visualize()

In [96]:
error_type_heatmap.visualize_table()


   Tag  B-MISC  I-MISC  B-ORG  O  I-LOC  B-LOC  I-ORG  B-PER  I-PER      Model
B-MISC       0       1     10 57      0      4      0      0      4 AraBERTv02
I-MISC       5       0      2 59      3     12     14      1      1 AraBERTv02
 B-ORG       7       0      0 49      4     19      1     15      0 AraBERTv02
     O      14      10     39  0      2     14     25     34      6 AraBERTv02
 I-LOC       0       0      0  8      0      7      0      0      0 AraBERTv02
 B-LOC       3       0      3 24      1      0      2      1      1 AraBERTv02
 I-ORG       0       2      9 50      5      2      0      2      6 AraBERTv02
 B-PER       4       1     12 44      0      8      2      0     33 AraBERTv02
 I-PER       1       1      1 20      1      0      5     35      0 AraBERTv02
B-MISC       0      11     39 30      0     25      1      7      0       BERT
I-MISC       5       0      1 21      8      0     17      2      1       BERT
 B-ORG      34       0      0 21      0     57     1

In [49]:
df = dash_data['ANERCorp_CamelLab_arabertv02'].analysis_data
df[(df['True Labels'] == 'B-LOC')&(df['Pred Labels'] != 'B-LOC')]['Pred Labels'].value_counts()
# df[(df['Pred Labels'] == 'B-LOC') & (df['True Labels'] == 'B-MISC')]

Pred Labels
O         24
B-MISC     3
B-ORG      3
I-ORG      2
B-PERS     1
I-PERS     1
I-LOC      1
Name: count, dtype: int64

In [22]:
dash_data['ANERCorp_CamelLab_arabertv02'].token_misclassifications

,B-MISC,I-MISC,B-ORG,O,I-LOC,B-LOC,I-ORG,B-PERS,I-PERS
B-MISC,0,1,10,57,0,4,0,0,4
I-MISC,5,0,2,59,3,12,14,1,1
B-ORG,7,0,0,49,4,19,1,15,0
O,14,10,39,0,2,14,25,34,6
I-LOC,0,0,0,8,0,7,0,0,0
B-LOC,3,0,3,24,1,0,2,1,1
I-ORG,0,2,9,50,5,2,0,2,6
B-PERS,4,1,12,44,0,8,2,0,33
I-PERS,1,1,1,20,1,0,5,35,0


In [56]:
dash_data['ANERCorp_CamelLab_arabertv02'].analysis_data['Error Type'].value_counts()

Error Type
No Errors            29005
Exclusion              311
Inclusion              144
Type                   124
Boundary                92
Type and Boundary       35
Name: count, dtype: int64